# Trabajo Práctico Nº3 - Big Data

### Mateo Servent, Joaquín Musich y Andres Cuellas

### Parte I: Análisis de la base de hogares y cálculo de pobreza

In [6]:
import pandas as pd

# Leer la base de datos
df = pd.read_excel('/Users/macbook/Desktop/UdeSa/Materias/Big data/Big-Data/TP3/usu_hogar_T123.xlsx')

df.describe().T

,count,mean,std,min,25%,50%,75%,max
ANO4,16815.0,2023.000000,0.000000,2023.0,2023.0,2023.0,2023.0,2023.0
TRIMESTRE,16815.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
NRO_HOGAR,16815.0,1.064347,1.177592,1.0,1.0,1.0,1.0,52.0
REALIZADA,16815.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
REGION,16815.0,35.439191,15.237237,1.0,40.0,42.0,43.0,44.0
...,...,...,...,...,...,...,...,...
VII1_2,16815.0,0.590009,2.756843,0.0,0.0,0.0,0.0,97.0
VII2_1,16815.0,70.012608,43.469641,1.0,3.0,98.0,98.0,98.0
VII2_2,16815.0,0.403449,3.397350,0.0,0.0,0.0,0.0,98.0
VII2_3,16815.0,0.103836,1.289136,0.0,0.0,0.0,0.0,98.0
